# 여행 미션-목적 비교 전처리
1. 필요한 라이브러리를 불러옵니다.
2. 데이터 경로를 확인하고 입출력 위치를 준비합니다.
3. 원본 데이터에서 미션 관련 파생 컬럼을 제거합니다.
4. `TRAVEL_MISSION_CHECK`와 `TRAIN_PURPOSE`를 정리해 중복과 공백을 없앱니다.
5. 두 컬럼을 비교해 미스매치 비율(`MISSION_PURPOSE_GAP_RATIO`)을 계산합니다.
6. 결과를 저장하고 주요 컬럼을 미리보기로 확인합니다.

In [ ]:
# 기본 라이브러리 임포트
import pandas as pd
from pathlib import Path

In [ ]:
# 데이터 파일과 출력 경로 찾기
run_dir = Path.cwd().resolve()
search_roots = [run_dir, run_dir.parent, run_dir.parent.parent]
input_path = None
output_dir = None
for root in search_roots:
    candidate = root / 'data' / 'training' / 'preprocessing' / 'final_travel_dataset.csv'
    if candidate.exists():
        input_path = candidate
        output_dir = root / 'data' / 'training' / 'final'
        break
if input_path is None:
    raise FileNotFoundError('final_travel_dataset.csv 위치를 찾을 수 없습니다.')
output_dir.mkdir(parents=True, exist_ok=True)
print(f'입력 파일 경로: {input_path}')
print(f'출력 디렉터리: {output_dir}')
output_path = output_dir / 'travel_insight_v1.csv'

In [ ]:
# 원본 데이터 불러오고 미션 파생 컬럼 제거하기
travel_df = pd.read_csv(input_path)
print('원본 데이터 모양:', travel_df.shape)
mission_cols = [col for col in travel_df.columns if col.startswith('MISSION_CD_')]
print('삭제할 미션 파생 컬럼 수:', len(mission_cols))
travel_df = travel_df.drop(columns=mission_cols)

In [ ]:
# ';'로 연결된 코드를 정렬된 리스트로 바꾸는 보조 함수
# 공백과 중복을 제거하고, 초보자도 보기 쉬운 형태로 반환합니다.
def parse_codes(raw):
    if pd.isna(raw):
        return []
    tokens = [token.strip() for token in str(raw).split(';') if token.strip()]
    return sorted(set(tokens))

# 문자열 컬럼을 정리해 다시 ';'로 연결합니다.
normalized_mission = travel_df['TRAVEL_MISSION_CHECK'].map(parse_codes)
normalized_purpose = travel_df['TRAIN_PURPOSE'].map(parse_codes)
travel_df['TRAVEL_MISSION_CHECK'] = normalized_mission.map(lambda codes: ';'.join(codes))
travel_df['TRAIN_PURPOSE'] = normalized_purpose.map(lambda codes: ';'.join(codes))

In [ ]:
# 미션 대비 목적의 미스매치 비율 계산 함수
# 예시: mission "1;22;7" vs purpose "2;4" -> mission_set={1,22,7}, purpose_set={2,4}
# 미션에서 목적에 없는 코드 3개 / 전체 미션 3개 = 1.0 (100% 미스매치)
def compute_gap(mission_codes, purpose_codes):
    if not mission_codes:
        return pd.NA
    mission_set = set(mission_codes)
    purpose_set = set(purpose_codes)
    gap_ratio = (len(mission_set - purpose_set)) / len(mission_set)
    return round(gap_ratio, 4)

travel_df['MISSION_PURPOSE_GAP_RATIO'] = [
    compute_gap(mission, purpose)
    for mission, purpose in zip(normalized_mission, normalized_purpose)
]

In [ ]:
# 컬럼 순서를 정리하고 CSV로 저장한 뒤 일부 결과를 확인합니다.
base_cols = [
    col for col in travel_df.columns
    if col not in {'TRAVEL_MISSION_CHECK', 'MISSION_PURPOSE_GAP_RATIO'}
]
ordered_cols = base_cols + ['TRAVEL_MISSION_CHECK', 'MISSION_PURPOSE_GAP_RATIO']
travel_df = travel_df[ordered_cols]
travel_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f'저장 완료: {output_path}')
print('마지막 5개 컬럼:', ordered_cols[-5:])
print(travel_df[['TRAVEL_MISSION_CHECK', 'TRAIN_PURPOSE', 'MISSION_PURPOSE_GAP_RATIO']].head())